In [3]:
!pip install pymystem3

!pip install pymorphy2

!pip install jsonlines

from pymystem3 import Mystem

import pymorphy2
from pymorphy2 import MorphAnalyzer

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import json

import jsonlines

import string

import collections
from collections import Counter

import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 8.2 MB 33.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=d4802cb1ca75998eaf5dcc4eaffc451be049d309cebce419550b5ff138b41880
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


We start with opening the text and converting it's type to string:

In [7]:
with open('capote.txt', 'r') as file:
    text_in = file.read().rstrip()

text1 = text_in.translate(str.maketrans('', '', string.punctuation))

pattern = r'[0-9]'
text = re.sub(pattern, '', text1)

First we lemmatize the text:

In [ ]:
m = Mystem()
lemmas = m.lemmatize(text)

text_out = open('mystem_capote.txt','w')
text_out.write(''.join(lemmas))
text_out.close()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


KeyboardInterrupt: ignored

Then we tokenize it:

In [8]:
tokens = nltk.word_tokenize(text)

Now we analyze the resulting words:

In [9]:
morph = MorphAnalyzer()

i = 0
lines = []

while i < len(tokens):
    ana = morph.parse(tokens[i])
    current = ana[0]
    d = {'lemma': current.normal_form, 'word': current.word, 'pos': current.tag.POS}
    lines.append(d)
    i += 1

with jsonlines.open('pymorphy_capote.jsonl', mode='w') as writer:
    for d in lines:
        writer.write(d)

**1) What percentage constitutes each pos? (E.g., for the verb, the number of verbs divided by the total number of words)**

To answer this question we need to make a list with every pos and it's number of entrances:

In [10]:
list_of_pos = []

for d in lines:
    pos = d.get('pos')
    list_of_pos.append(pos)

print(Counter(list_of_pos))

Counter({'NOUN': 5108, 'VERB': 3389, 'NPRO': 2877, 'CONJ': 2363, 'PREP': 2274, 'ADJF': 2032, 'ADVB': 1429, 'PRCL': 1320, 'INFN': 670, 'GRND': 221, 'ADJS': 152, 'NUMR': 144, 'PRTF': 139, 'PRED': 127, 'COMP': 115, 'PRTS': 99, None: 88, 'INTJ': 40})


And here is the number of all words in the text:

In [11]:
print(len(lines))

22587


Now all we need is to print out the percentage for every pos (I decided to display 3 digits after the dot because otherwise percentage of some pos comes out as 0):

In [14]:
y = [[x,list_of_pos.count(x)] for x in set(list_of_pos)]

for i in y:
    print(i[0], " = ", round(i[1]/len(lines), 3))

PRTS  =  0.004
PREP  =  0.101
ADVB  =  0.063
None  =  0.004
NOUN  =  0.226
ADJS  =  0.007
PRTF  =  0.006
PRCL  =  0.058
INTJ  =  0.002
NUMR  =  0.006
INFN  =  0.03
GRND  =  0.01
ADJF  =  0.09
PRED  =  0.006
VERB  =  0.15
COMP  =  0.005
NPRO  =  0.127
CONJ  =  0.105


The strange category 'None' represents all the numbers, foreign words, etc. -- I decided to keep them since they are a legit part of text (and either way there are not so many of these).

---

**2) Print out top-20 verbs and adverbs.**

To answer this question we need to make a list with of every verb and another list of every adverb (it's not clear whether I need to prin out a top-20 of verbs and adverbs or a top-20 of verbs and top-20 of adverbs so I decided to do to do separate tops) and then to sort out top-20:

In [15]:
from scipy.special import k0
list_of_verbs = []
list_of_adverbs = []

for d in lines:
    if d["pos"] == 'VERB':
        verb = d["lemma"]
        list_of_verbs.append(verb)

print(Counter(list_of_verbs).most_common(20))

for d in lines:
    if d["pos"] == 'ADVB':
        adverb = d["lemma"]
        list_of_adverbs.append(adverb)

print(Counter(list_of_adverbs).most_common(20))

[('быть', 293), ('сказать', 146), ('мочь', 106), ('знать', 69), ('говорить', 49), ('хотеть', 47), ('любить', 40), ('казаться', 39), ('думать', 36), ('стать', 33), ('голайтлить', 30), ('видеть', 27), ('спросить', 22), ('дать', 19), ('идти', 19), ('жить', 18), ('взять', 18), ('стоять', 17), ('увидеть', 17), ('стоить', 16)]
[('только', 72), ('ещё', 68), ('там', 47), ('уже', 40), ('потому', 36), ('потом', 35), ('где', 34), ('теперь', 33), ('очень', 30), ('пока', 30), ('здесь', 20), ('тут', 20), ('далее', 20), ('всегда', 19), ('совсем', 19), ('ничего', 17), ('снова', 16), ('тогда', 16), ('несколько', 15), ('вдруг', 15)]


Next task is as follows:
**Find top-25 bigrams and trigrams for your text (use nltk.bigrams), use only lemmas, get rid of the punctuation. Comment shortly on the results.**

In [17]:
with open('mystem_capote.txt', 'r') as file:
    text_new = file.read().rstrip()

bigrm = list(nltk.bigrams(text_new.split()))
print(Counter(bigrm).most_common(25))

trigrm = list(nltk.trigrams(text_new.split()))
print(Counter(trigrm).most_common(25))

[(('я', 'не'), 63), (('что', 'я'), 53), (('сказать', 'она'), 51), (('у', 'я'), 39), (('не', 'быть'), 37), (('и', 'я'), 34), (('потому', 'что'), 34), (('она', 'и'), 30), (('и', 'не'), 30), (('это', 'быть'), 29), (('не', 'мочь'), 29), (('она', 'не'), 29), (('сказать', 'он'), 27), (('что', 'она'), 25), (('я', 'в'), 25), (('я', 'и'), 24), (('то', 'что'), 24), (('он', 'не'), 24), (('когда', 'я'), 24), (('что', 'ты'), 24), (('джо', 'белл'), 23), (('он', 'и'), 23), (('что', 'он'), 22), (('у', 'она'), 22), (('она', 'быть'), 21)]
[(('сказать', 'она', 'и'), 11), (('о', 'д', 'берман'), 11), (('так', 'и', 'не'), 7), (('в', 'конец', 'конец'), 7), (('что', 'я', 'не'), 7), (('сказать', 'он', 'и'), 6), (('с', 'тот', 'пора'), 6), (('к', 'то', 'же'), 5), (('один', 'из', 'они'), 5), (('бар', 'джо', 'белла'), 5), (('не', 'знать', 'что'), 5), (('я', 'за', 'рука'), 5), (('мадам', 'сапфия', 'спанелла'), 5), (('то', 'что', 'я'), 5), (('не', 'мочь', 'быть'), 5), (('я', 'не', 'мочь'), 5), (('в', 'то', 'что'), 5

As we can see practically all bigrams are deprived of sense on their own but trigrams look more interesting: for example one of the most common trigrams is (**в**, **конец**, **конец**) which clearly refers to **в конце концов** -- it says a lot about the text guess. Another popular trigram is бар Джо Белла -- this one speaks for itself too.

---

The last task is as follows:
**Take 3-8 sentences from the original text and substitute some morphological information, for example, change the tense of the verbs, the number of the nouns, e.g, the original Слон подарил мартышке цветы should become Слоны подарят мартышкам цветок.**

In [19]:
extra = 'Однажды поздно ночью я проснулся от того, что мистер Юниоши что-то кричал в пролет лестницы. Он жил на верхнем этаже, и голос его, строгий и сердитый, разносился по ' +\
'всему дому: - Мисс Голайтли! Я должен протестовать! Ребячливый, беззаботно-веселый голос ответил снизу: - Миленький, простите! Я опять потеряла этот дурацкий ключ. ' +\
'- Пожалуйста, не надо звонить в мой звонок. - Пожалуйста, сделайте себе ключ! - Да я их всё время теряю. - Я работаю, я должен спать, - кричал мистер Юниоши. - А вы всё ' +\
'время звоните в мой звонок! - Миленький вы мой, ну зачем вы сердитесь? Я больше не буду. Пожалуйста, не сердитесь. - Голос приближался, она поднималась по лестнице.'

First we tokenize the text in order to deal with separate words:

In [20]:
tokens = nltk.word_tokenize(extra)

m = Mystem()
morph = MorphAnalyzer()

extra_new = []

Now we transform the text to the text of the same meaning but in plural form.

The general idea is simple: we need to switch nouns, pronouns, verbs, adjectives (including the short ones) to their plural forms and leave the others as they are. Also in case of verbs we need to maintain their tense and mood.

The code is devided in 4 major chunks:

**1. Exceptions.**

There are 3 kinds of issues with some words -- proper nouns, suiting parses and something I can't even name. **The first issue** is that proper nouns (in my case these are *Юниоши*, *Голайтли*) don't have plural forms but there seems to be no way to distinguish them from common nouns automatically. **The second issue** is that from time to time the most probable parse (i.e. ana[0]) is not the correct one (in my case these are *того*, *лестницы*) -- and again I can't think of a way to automatically verify whether ana[0] is the correct parse for each word. As for **the third issue** -- sometimes there are words that can be changed to their plural forms but shouldn't be in this particular case (in my case this is *время*) -- now this could be solved with syntactic analysis so it is worth looking into the issue;

**2. Pronouns**

The problem with pronouns is they **should** be changed to the plural forms but at the same time, for example, *мы* is not a plural form for *я* strictly speaking. For this reason I had no other way but to write down every option manualy;

**3. Verbs, nouns, adjectives, short adjectives**

In this chunk of code everything is quite straightforward: we switch words fitting the condition to their plural forms (at the same time keeping same tense and mood for verbs);

**4. The rest**

Every other token should be left as it is -- which is exactly what I do!

**And here we go:**

In [5]:
for i in tokens:
    ana = morph.parse(i)
    wrd = ana[0]
    #several exception that I couldn't figure out
    if wrd.word == 'юниоши' or wrd.word == 'голайтли' or wrd.word == 'время' or wrd.word == 'того' or wrd.word == 'лестницы':
        if wrd.word == 'юниоши':
            text_new.append('Юниоши')
        elif wrd.word == 'голайтли':
            text_new.append('Голайтли')
        elif wrd.word == 'время':
            text_new.append('время')
        elif wrd.word == 'того':
            text_new.append('того')
        elif wrd.word == 'лестницы':
            text_new.append('лестниц')
    else:
        #pronouns that need to be switched to plural forms manualy 
        if wrd.tag.POS == 'NPRO':
            if wrd.word == 'я':
                pl = 'мы'
            elif wrd.word == 'ты':
                pl = 'вы'
            elif wrd.word == 'он' or wrd.word == 'она' or wrd.word == 'оно':
                pl = 'они'
            elif wrd.word == 'свой' or wrd.word == 'своя' or wrd.word == 'своё':
                pl = 'свои'
            elif wrd.word == 'мой' or wrd.word == 'моя' or wrd.word == 'моё':
                pl = 'наши'
            elif wrd.word == 'твой' or wrd.word == 'твоя' or wrd.word == 'твоё':
                pl = 'ваши'
            elif wrd.word == 'его' or wrd.word == 'её':
                pl = 'их'
            elif wrd.word == 'себе':
                pl = 'себе'
            else:
                #pronouns already in plural form
                pl = wrd.word
            text_new.append(pl)
        else:
            #major text change happens here
            if wrd.tag.POS == 'VERB' or wrd.tag.POS == 'NOUN' or wrd.tag.POS == 'ADJF' or wrd.tag.POS == 'ADJS':
                #paying attention to verbs and their tags
                if wrd.tag.POS == 'VERB':
                    if wrd.tag.tense == 'past':
                        pl = wrd.inflect({'plur', 'past'})
                    elif wrd.tag.tense == 'pres':
                        pl = wrd.inflect({'plur', 'pres'})
                    elif wrd.tag.tense == 'futr':
                        pl = wrd.inflect({'plur', 'futr'})
                    elif wrd.tag.mood == 'impr' and wrd.tag.number == 'sing':
                        pl = wrd.inflect({'impr', 'plur'})
                    elif wrd.tag.mood == 'impr' and wrd.tag.number == 'plur':
                        pl = wrd.inflect({'impr', 'plur'})
                else:
                    #nouns and adjectives are less demanding
                    pl = wrd.inflect({'plur'})
                text_new.append(pl.word)
            else:
                #and finally all the rest tokens need no change
                pl = wrd.word
                text_new.append(wrd.word)

And now I just need to print out the text and the result of the program:

In [6]:
print('Text:', extra, end = '\n\n')
print(' '.join(extra_new), end = '\n\n')

Text: Однажды поздно ночью я проснулся от того, что мистер Юниоши что-то кричал в пролет лестницы. Он жил на верхнем этаже, и голос его, строгий и сердитый, разносился по всему дому: - Мисс Голайтли! Я должен протестовать! Ребячливый, беззаботно-веселый голос ответил снизу: - Миленький, простите! Я опять потеряла этот дурацкий ключ. - Пожалуйста, не надо звонить в мой звонок. - Пожалуйста, сделайте себе ключ! - Да я их всё время теряю. - Я работаю, я должен спать, - кричал мистер Юниоши. - А вы всё время звоните в мой звонок! - Миленький вы мой, ну зачем вы сердитесь? Я больше не буду. Пожалуйста, не сердитесь. - Голос приближался, она поднималась по лестнице.

однажды поздно ночью мы проснулись от того , что мистеры Юниоши что-то кричали в пролёты лестниц . они жили на верхних этажах , и голоса их , строгие и сердитые , разносились по всем домам : - мисс Голайтли ! мы должны протестовать ! ребячливые , беззаботно-весёлые голоса ответили снизу : - миленькие , простите ! мы опять потеря